In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset from an Excel file
data = pd.read_excel('t5_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
models = {
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "MLP": MLPClassifier(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=0),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
}

# Evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    results[name] = {
        'Accuracy': round(accuracy, 4),
        'Precision': round(precision, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4)
    }

# Print the results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model: SVM
  Accuracy: 0.6042
  Precision: 0.6072
  Recall: 0.5875
  F1 Score: 0.5907
Model: KNN
  Accuracy: 0.5357
  Precision: 0.5401
  Recall: 0.5222
  F1 Score: 0.5271
Model: Decision Tree
  Accuracy: 0.4494
  Precision: 0.4486
  Recall: 0.4449
  F1 Score: 0.4464
Model: Gaussian Naive Bayes
  Accuracy: 0.4643
  Precision: 0.4618
  Recall: 0.4886
  F1 Score: 0.4613
Model: Random Forest
  Accuracy: 0.5595
  Precision: 0.5654
  Recall: 0.5437
  F1 Score: 0.5477
Model: Logistic Regression
  Accuracy: 0.5357
  Precision: 0.5472
  Recall: 0.5346
  F1 Score: 0.5398
Model: MLP
  Accuracy: 0.5833
  Precision: 0.5847
  Recall: 0.574
  F1 Score: 0.5779
Model: XGBoost
  Accuracy: 0.5714
  Precision: 0.5788
  Recall: 0.5628
  F1 Score: 0.5681
Model: CatBoost
  Accuracy: 0.5833
  Precision: 0.5854
  Recall: 0.5671
  F1 Score: 0.571
Model: AdaBoost
  Accuracy: 0.5298
  Precision: 0.5575
  Recall: 0.5087
  F1 Score: 0.5156
Model: Extra Trees
  Accuracy: 0.5506
  Precision: 0.5477
  Recall: 0.5301


In [ ]:


# Define the SVM model
svm = SVC()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Applicable only for polynomial kernel
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

# Get the best estimator
best_svm = random_search.best_estimator_

# Perform cross-validation on the best model
cv_score = cross_validate(best_svm, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)




Best Parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'C': 1}
Evaluation Metrics: {'Accuracy Mean': 0.622, 'Accuracy STD': 0.0398, 'Precision Mean': 0.6381, 'Precision STD': 0.0389, 'Recall Mean': 0.6183, 'Recall STD': 0.0431, 'F1 Mean': 0.6239, 'F1 STD': 0.0422}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset from an Excel file
data = pd.read_excel('t5_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_gb = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_gb, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Parameters: {'subsample': 1.0, 'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.3}
Evaluation Metrics: {'Accuracy Mean': 0.6087, 'Accuracy STD': 0.0332, 'Precision Mean': 0.6185, 'Precision STD': 0.0316, 'Recall Mean': 0.6088, 'Recall STD': 0.0387, 'F1 Mean': 0.6104, 'F1 STD': 0.0342}


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Extra Trees model
extra_trees_model = ExtraTreesClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [False, True]
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=extra_trees_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_extra_trees = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_extra_trees, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
40 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Evaluation Metrics: {'Accuracy Mean': 0.5953, 'Accuracy STD': 0.0281, 'Precision Mean': 0.6106, 'Precision STD': 0.0223, 'Recall Mean': 0.5892, 'Recall STD': 0.032, 'F1 Mean': 0.5939, 'F1 STD': 0.03}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier

# Define the kNN model
knn = KNeighborsClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_knn = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_knn, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'weights': 'distance', 'n_neighbors': 14, 'metric': 'manhattan'}
Evaluation Metrics: {'Accuracy Mean': 0.5834, 'Accuracy STD': 0.0428, 'Precision Mean': 0.5964, 'Precision STD': 0.0416, 'Recall Mean': 0.5824, 'Recall STD': 0.0465, 'F1 Mean': 0.5841, 'F1 STD': 0.0442}


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # For handling class imbalance

# Optional: Scaling the features (this isn't strictly necessary for Decision Trees but may help)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Optional: Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the Decision Tree model
dtree = DecisionTreeClassifier(random_state=42, class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 10],
    'max_features': [None, 'sqrt', 'log2'],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dtree,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=StratifiedKFold(n_splits=10),  # Use StratifiedKFold to preserve class distribution
    scoring='f1_macro',  # F1 macro to handle imbalanced classes
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

# Get the best Decision Tree model
best_dtree = random_search.best_estimator_

# Perform cross-validation on the best model
cv_score = cross_validate(best_dtree, X_train_resampled, y_train_resampled, cv=StratifiedKFold(n_splits=10),
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini'}
Evaluation Metrics: {'Accuracy Mean': 0.5591, 'Accuracy STD': 0.0509, 'Precision Mean': 0.5616, 'Precision STD': 0.0496, 'Recall Mean': 0.5591, 'Recall STD': 0.0509, 'F1 Mean': 0.5554, 'F1 STD': 0.0459}


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

# No hyperparameters to tune, so we can fit and evaluate directly
nb.fit(X_train, y_train)

# Perform cross-validation on the model
cv_score = cross_validate(nb, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Train the Naive Bayes model
nb = GaussianNB(var_smoothing=1e-9)  # Apply Laplace smoothing
nb.fit(X_train_resampled, y_train_resampled)

# Step 4: Evaluate using Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10)

cv_score = cross_validate(nb, X_train_resampled, y_train_resampled, cv=skf,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Naive Bayes Evaluation Metrics:", results)



Naive Bayes Evaluation Metrics: {'Accuracy Mean': 0.5428, 'Accuracy STD': 0.0327, 'Precision Mean': 0.5388, 'Precision STD': 0.0301, 'Recall Mean': 0.5428, 'Recall STD': 0.0327, 'F1 Mean': 0.5343, 'F1 STD': 0.0311}


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Scaling (important for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE (if needed)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Define the AdaBoost model with a shallow decision tree as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

# Step 4: Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds (trees)
    'learning_rate': [0.01, 0.1, 1.0],  # How much each estimator contributes
    'estimator__max_depth': [3, 5, 7]  # Max depth of the decision tree (base estimator)
}

# Step 5: Set up RandomizedSearchCV with StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit the model with the best parameters
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters found
print("AdaBoost Best Parameters:", random_search.best_params_)

# Step 7: Perform cross-validation on the best AdaBoost model
best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
# Step 8: Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("AdaBoost Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Best Parameters: {'n_estimators': 200, 'learning_rate': 1.0, 'estimator__max_depth': 5}


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

AdaBoost Evaluation Metrics: {'Accuracy Mean': 0.6623, 'Accuracy STD': 0.0639, 'Precision Mean': 0.676, 'Precision STD': 0.0528, 'Recall Mean': 0.6623, 'Recall STD': 0.0639, 'F1 Mean': 0.6632, 'F1 STD': 0.0584}


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

# Define the CatBoost model
catboost = CatBoostClassifier(silent=True)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'iterations': [50, 100, 200],
    'depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=20,
    cv=10,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("CatBoost Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_catboost = random_search.best_estimator_
cv_score = cross_validate(best_catboost, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("CatBoost Evaluation Metrics:", results)


KeyboardInterrupt: 

In [ ]:
from sklearn.neural_network import MLPClassifier

# Define the MLP model
mlp = MLPClassifier(max_iter=2000, random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_distributions,
    n_iter=5,
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("MLP Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_mlp = random_search.best_estimator_
cv_score = cross_validate(best_mlp, X_train, y_train, cv=5, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("MLP Evaluation Metrics:", results)


MLP Best Parameters: {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Evaluation Metrics: {'Accuracy Mean': 0.5982, 'Accuracy STD': 0.0269, 'Precision Mean': 0.5998, 'Precision STD': 0.0291, 'Recall Mean': 0.6051, 'Recall STD': 0.0323, 'F1 Mean': 0.5998, 'F1 STD': 0.0279}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset from an Excel file
data = pd.read_excel('t5_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Define the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5,10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt'],
    'bootstrap': [True,False],
    'class_weight': ['balanced']
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=5,
    cv=StratifiedKFold(n_splits=5),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_rf = random_search.best_estimator_

# Perform cross-validation on the best Random Forest model
cv_score = cross_validate(best_rf, X_train, y_train, cv=5,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'class_weight': 'balanced', 'bootstrap': True}
Evaluation Metrics: {'Accuracy Mean': 0.6027, 'Accuracy STD': 0.0173, 'Precision Mean': 0.6122, 'Precision STD': 0.0135, 'Recall Mean': 0.6007, 'Recall STD': 0.0226, 'F1 Mean': 0.604, 'F1 STD': 0.0187}


In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the Logistic Regression model
log_reg = LogisticRegression(solver='liblinear', random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_distributions,
    n_iter=5,
    cv=10,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Logistic Regression Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_log_reg = random_search.best_estimator_
cv_score = cross_validate(best_log_reg, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Logistic Regression Evaluation Metrics:", results)


Logistic Regression Best Parameters: {'penalty': 'l2', 'C': 0.01}
Logistic Regression Evaluation Metrics: {'Accuracy Mean': 0.6035, 'Accuracy STD': 0.0501, 'Precision Mean': 0.6053, 'Precision STD': 0.0509, 'Recall Mean': 0.6114, 'Recall STD': 0.0507, 'F1 Mean': 0.6034, 'F1 STD': 0.0492}


In [ ]:
#pca
#pca
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Load the dataset
data = pd.read_excel('t5_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Scale the features
X_scaled = StandardScaler().fit_transform(X)

# Apply PCA to retain 95% of variance
pca = PCA(n_components=0.9999)
X_pca = pca.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_gb = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_gb, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
Evaluation Metrics: {'Accuracy Mean': 0.538, 'Accuracy STD': 0.0388, 'Precision Mean': 0.5505, 'Precision STD': 0.0431, 'Recall Mean': 0.5274, 'Recall STD': 0.038, 'F1 Mean': 0.5321, 'F1 STD': 0.0393}


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Extra Trees model
extra_trees_model = ExtraTreesClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [False, True]
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=extra_trees_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_extra_trees = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_extra_trees, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
40 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
Evaluation Metrics: {'Accuracy Mean': 0.5246, 'Accuracy STD': 0.0371, 'Precision Mean': 0.5758, 'Precision STD': 0.0534, 'Recall Mean': 0.4908, 'Recall STD': 0.0392, 'F1 Mean': 0.4894, 'F1 STD': 0.0462}
